In [1]:
import torch
from torch import nn

from torch.optim import lr_scheduler

from torch.utils.data import random_split,Dataset,DataLoader

import torch.nn.functional as F
import torch.nn.init as init

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import time
import copy
import random
import pickle
import tarfile

import argparse
import pandas as pd
import dgl
import torch
import torch.nn.functional as F
import collections
from scipy.sparse import csr_matrix, vstack, save_npz
from sklearn.decomposition import PCA
from pathlib import Path
import numpy as np
from pprint import pprint
import json
from collections import Counter

import time


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Using backend: pytorch


In [2]:
from dgl.contrib.sampling import NeighborSampler

In [3]:
import dgl.function as fn

In [4]:
torch.version

<module 'torch.version' from '/home/chenhuaguan/.conda/envs/python3.6/lib/python3.6/site-packages/torch/version.py'>

# Create Graph

In [5]:
device = torch.device('cpu')

In [6]:
data_gene = pd.read_csv("data/human_Fetal_eye1880_data.csv",index_col = 0)

In [7]:
data_gene

,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_1871,C_1872,C_1873,C_1874,C_1875,C_1876,C_1877,C_1878,C_1879,C_1880
A2M,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M-AS1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2ML1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A4GALT,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAAS,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TUT4,0.0,2.274021,0.0,0.0,0.0,0.0,0.0,0.0,1.903265,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RTL4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TUT7,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNF788P,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
id2gene = data_gene.index.values.tolist()
id2gene.sort()

In [9]:
data_cell = pd.read_csv("data/human_Fetal_eye1880_celltype.csv", dtype=np.str, header=0,index_col=0)

In [10]:
cell = set()
cell_type_list = []
data_cell['Cell_type'] = data_cell['Cell_type'].map(str.strip)
cell_types = set(data_cell.values[:, 1])
cell_type_list.extend(data_cell.values[:, 1].tolist())
id2label = list(cell_types)
label_statistics = dict(collections.Counter(cell_type_list))
total_cell = sum(label_statistics.values())

In [11]:
for label, num in label_statistics.items():
    if num / total_cell <= 0.005:
        id2label.remove(label)  # remove exclusive labels
gene2id = {gene: idx for idx, gene in enumerate(id2gene)}
num_genes = len(id2gene)
# prepare unified labels
num_labels = len(id2label)
label2id = {label: idx for idx, label in enumerate(id2label)}

In [12]:
graph = dgl.DGLGraph()

gene_ids = torch.arange(num_genes, dtype=torch.int32, device=device).unsqueeze(-1)
graph.add_nodes(num_genes, {'id': gene_ids})
all_labels = []
matrices = []
num_cells = 0
cell2type = pd.read_csv("data/human_Fetal_eye1880_celltype.csv",index_col=0)
cell2type.columns = ['cell', 'type']
cell2type['type'] = cell2type['type'].map(str.strip)
cell2type['id'] = cell2type['type'].map(label2id)
filter_cell = np.where(pd.isnull(cell2type['id']) == False)[0]
cell2type = cell2type.iloc[filter_cell]
all_labels += cell2type['id'].tolist()
df = pd.read_csv("data/human_Fetal_eye1880_data.csv",index_col = 0)
df = df.transpose(copy=True) 

In [13]:
df = df.iloc[filter_cell]
df = df.rename(columns=gene2id)
col = [c for c in df.columns if c in gene2id.values()]
df = df[col]
arr = df.to_numpy()
row_idx, col_idx = np.nonzero(arr > 0)  # intra-dataset index
non_zeros = arr[(row_idx, col_idx)]  # non-zero values
cell_idx = row_idx + graph.number_of_nodes()  # cell_index
gene_idx = df.columns[col_idx].astype(int).tolist()  # gene_index
info_shape = (len(df), num_genes)
info = csr_matrix((non_zeros, (row_idx, gene_idx)), shape=info_shape)
matrices.append(info)

num_cells += len(df)

In [14]:
ids = torch.tensor([-1] * len(df), dtype=torch.int32, device=device).unsqueeze(-1)
graph.add_nodes(len(df), {'id': ids})
graph.add_edges(cell_idx, gene_idx,
                {'weight': torch.tensor(non_zeros, dtype=torch.float32, device=device).unsqueeze(1)})
graph.add_edges(gene_idx, cell_idx,
                {'weight': torch.tensor(non_zeros, dtype=torch.float32, device=device).unsqueeze(1)})

In [15]:
sparse_feat = vstack(matrices).toarray()  # cell-wise  (cell, gene)
gene_pca = PCA(400, random_state=10086).fit(sparse_feat.T)
gene_feat = gene_pca.transform(sparse_feat.T)
gene_evr = sum(gene_pca.explained_variance_ratio_) * 100
sparse_feat = sparse_feat / (np.sum(sparse_feat, axis=1, keepdims=True) + 1e-6)
# use weighted gene_feat as cell_feat
cell_feat = sparse_feat.dot(gene_feat)
gene_feat = torch.from_numpy(gene_feat)  # use shared storage
cell_feat = torch.from_numpy(cell_feat)

graph.ndata['features'] = torch.cat([gene_feat, cell_feat], dim=0).type(torch.float).to(device)
labels = torch.tensor([-1] * num_genes + all_labels, dtype=torch.long, device=device)  # [gene_num+train_num]
per = np.random.permutation(range(num_genes, num_genes + num_cells))
test_ids = torch.tensor(per[:int(num_cells // ((1 - 0.2) / 0.2 + 1))]).to(device)
train_ids = torch.tensor(per[int(num_cells // ((1 - 0.2) / 0.2 + 1)):]).to(device)

In [16]:
in_degrees = graph.in_degrees()
for i in range(graph.number_of_nodes()):
    src, dst, in_edge_id = graph.in_edges(i, form='all')
    if src.shape[0] == 0:
        continue
    edge_w = graph.edata['weight'][in_edge_id]
    graph.edata['weight'][in_edge_id] = in_degrees[i] * edge_w / torch.sum(edge_w)

In [17]:
# add self-loop
graph.add_edges(graph.nodes(), graph.nodes(),
                {'weight': torch.ones(graph.number_of_nodes(), dtype=torch.float, device=device).unsqueeze(1)})
graph.readonly(True)

# GNN

## NodeUpdate

In [18]:
class NodeUpdate(nn.Module):
    def __init__(self, in_feats, out_feats, activation=None, norm=None):
        super(NodeUpdate, self).__init__()
        self.fc_neigh = nn.Linear(in_features=in_feats, out_features=out_feats)
        self.activation = activation
        self.norm = norm
        nn.init.xavier_uniform_(self.fc_neigh.weight, gain=nn.init.calculate_gain('relu'))

    def forward(self, node):
        h_neigh = node.data['neigh']
        h_neigh = self.fc_neigh(h_neigh)
        if self.activation is not None:
            h_neigh = self.activation(h_neigh)
        if self.norm is not None:
            h_neigh = self.norm(h_neigh)
        return {'activation': h_neigh}

## GNN Model

In [19]:
class GNN(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes, n_layers, gene_num, activation=None, norm=None, dropout=0.0):
        super(GNN, self).__init__()
        self.n_layers = n_layers
        self.gene_num = gene_num
        if dropout != 0:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = None
        self.layers = nn.ModuleList()
        self.layers.append(NodeUpdate(in_feats=in_feats, out_feats=n_hidden, activation=activation, norm=norm))
        for _ in range(n_layers - 1):
            self.layers.append(NodeUpdate(in_feats=n_hidden, out_feats=n_hidden, activation=activation, norm=norm))

        # [gene_num] is alpha of gene-gene, [gene_num+1] is alpha of cell-cell self loop
        self.alpha = nn.Parameter(torch.tensor([1] * (self.gene_num + 2), dtype=torch.float32).unsqueeze(-1))
        self.linear = nn.Linear(n_hidden, n_classes)
        nn.init.xavier_uniform_(self.linear.weight, gain=nn.init.calculate_gain('relu'))

    def message_func(self, edges: dgl.udf.EdgeBatch):
        number_of_edges = edges.src['h'].shape[0]
        indices = np.expand_dims(np.array([self.gene_num + 1] * number_of_edges, dtype=np.int32), axis=1)
        src_id, dst_id = edges.src['id'].cpu().numpy(), edges.dst['id'].cpu().numpy()
        indices = np.where((src_id >= 0) & (dst_id < 0), src_id, indices)  # gene->cell
        indices = np.where((dst_id >= 0) & (src_id < 0), dst_id, indices)  # cell->gene
        indices = np.where((dst_id >= 0) & (src_id >= 0), self.gene_num, indices)  # gene-gene
        h = edges.src['h'] * self.alpha[indices.squeeze()]
        # return {'m': h}
        return {'m': h * edges.data['weight']}

    def forward(self, nf: dgl.NodeFlow):
        nf.layers[0].data['activation'] = nf.layers[0].data['features']
        for i, layer in enumerate(self.layers):
            h = nf.layers[i].data.pop('activation')
            if self.dropout:
                h = self.dropout(h)
            nf.layers[i].data['h'] = h
            nf.block_compute(i, self.message_func, fn.mean('m', 'neigh'), layer)
        h = nf.layers[-1].data.pop('activation')
        h = self.linear(h)
        return h

    def evaluate(self, nf: dgl.NodeFlow):
        def message_func(edges: dgl.EdgeBatch):
            # edges.src['h']： (number of edges, feature dim)
            number_of_edges = edges.src['h'].shape[0]
            indices = np.expand_dims(np.array([self.gene_num + 1] * number_of_edges, dtype=np.int32), axis=1)
            src_id, dst_id = edges.src['id'].cpu().numpy(), edges.dst['id'].cpu().numpy()
            indices = np.where((src_id >= 0) & (dst_id < 0), src_id, indices)  # gene->cell
            indices = np.where((dst_id >= 0) & (src_id < 0), dst_id, indices)  # cell->gene
            indices = np.where((dst_id >= 0) & (src_id >= 0), self.gene_num, indices)  # gene-gene
            h = edges.src['h'].cpu() * self.alpha[indices.squeeze()]
            return {'m': h * edges.data['weight'].cpu()}

        nf.layers[0].data['activation'] = nf.layers[0].data['features'].cpu()
        for i, layer in enumerate(self.layers):
            h = nf.layers[i].data.pop('activation')
            if self.dropout:
                h = self.dropout(h)
            nf.layers[i].data['h'] = h
            nf.block_compute(i, message_func, fn.mean('m', 'neigh'), layer)
        h = nf.layers[-1].data.pop('activation')
        h = self.linear(h)
        return h

# Adam_Train

In [20]:
class Adam_Trainer:
    def __init__(self, device_train,gnn_model,lr,weight_decay,Adam_epochs,batch_size,
                num_cells,
                num_genes,
                num_labels,
                graph,
                train_ids,
                test_ids,
                labels):
        
        self.device = device_train
        
        self.num_cells = num_cells
        self.num_genes = num_genes
        self.num_labels = num_labels
        
        self.graph = graph
        
        self.graph.readonly(True)
        
        self.train_ids = train_ids
        self.test_ids = test_ids
        self.labels = labels 
        
        self.labels = self.labels.to(self.device)
        
        self.model = gnn_model
        
        self.lr = lr
        
        self.weight_decay = weight_decay
        
        self.epochs = Adam_epochs
        
        self.batch_size = batch_size

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr,
                                          weight_decay=self.weight_decay)
        
        self.loss_fn = nn.CrossEntropyLoss(reduction='sum')
        
        self.num_neighbors = self.num_cells + self.num_genes

    def fit(self):
        max_test_acc, _train_acc, _epoch = 0, 0, 0
        for epoch in range(self.epochs):
            loss = self.train()
            train_correct, train_unsure = self.evaluate(self.train_ids, 'train')
            train_acc = train_correct / len(self.train_ids)
            test_correct, test_unsure = self.evaluate(self.test_ids, 'test')
            test_acc = test_correct / len(self.test_ids)
            if max_test_acc <= test_acc:
                final_test_correct_num = test_correct
                final_test_unsure_num = test_unsure
                _train_acc = train_acc
                _epoch = epoch
                max_test_acc = test_acc
                self.save_model()
            print(
                f">>>>Epoch {epoch+1:04d}: Train Acc {train_acc:.4f}, Loss {loss / len(self.train_ids):.4f}, Test correct {test_correct}, "
                f"Test unsure {test_unsure}, Test Acc {test_acc:.4f}")
            if train_acc == 1:
                break

        #print(f"---{self.params.species} {self.params.tissue} Best test result:---")
        print(f"Epoch {_epoch+1:04d}, Train Acc {_train_acc:.4f}, Test Correct Num {final_test_correct_num}, Test Total Num {len(self.test_ids)}, Test Unsure Num {final_test_unsure_num}, Test Acc {final_test_correct_num / len(self.test_ids):.4f}")

    def train(self):
        self.model.train()
        total_loss = 0
        for batch, nf in enumerate(NeighborSampler(g=self.graph,
                                                   batch_size=self.batch_size,
                                                   expand_factor=self.num_neighbors,
                                                   num_hops=1,
                                                   neighbor_type='in',
                                                   shuffle=True,
                                                   num_workers=8,
                                                   seed_nodes=self.train_ids.long())):
            nf.copy_from_parent()  # Copy node/edge features from the parent graph.
            logits = self.model(nf)
            batch_nids = nf.layer_parent_nid(-1).type(torch.long).to(device=self.device)
            loss = self.loss_fn(logits, self.labels[batch_nids])
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            total_loss += loss.item()

        return total_loss

    def evaluate(self, ids, type='test'):
        self.model.eval()
        total_correct, total_unsure = 0, 0
        for nf in NeighborSampler(g=self.graph,
                                  batch_size=self.batch_size,
                                  expand_factor=self.num_cells + self.num_genes,
                                  num_hops=1,
                                  neighbor_type='in',
                                  shuffle=True,
                                  num_workers=8,
                                  seed_nodes=ids.long()):
            nf.copy_from_parent()  # Copy node/edge features from the parent graph.
            with torch.no_grad():
                logits = self.model(nf).cpu()
            batch_nids = nf.layer_parent_nid(-1).type(torch.long)
            logits = nn.functional.softmax(logits, dim=1).numpy()
            label_list = self.labels.cpu()[batch_nids]
            for pred, label in zip(logits, label_list):
                max_prob = pred.max().item()
                if max_prob < 2 / num_labels:
                    total_unsure += 1
                elif pred.argmax().item() == label:
                    total_correct += 1

        return total_correct, total_unsure

    def save_model(self):
        state = {
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict()
        }

        torch.save(state, "GA_Model.pth")

# GA

In [21]:
def crossover_and_mutation(parents, sigma=0.05):

    
    base_sd = parents[0].state_dict()
    keys = base_sd                    # use all layers to be affected
    
    # Sum of the weights of the parent
    for i in range(1, len(parents)):
        parent_sd = parents[i].state_dict()
        for key in keys:
            base_sd[key] = base_sd[key] + parent_sd[key]
            
    
    # Average and add mutation
    num_parents = len(parents)
    
    for key in keys:
        
        tensor_size = base_sd[key].size()
        random_tensor = torch.normal(mean=0.0, std=sigma, size=tensor_size)
        
        base_sd[key] = (base_sd[key] / num_parents) + random_tensor
    
    # create offspring
    offspring = GNN(in_feats=in_feats,
                         n_hidden=n_hidden,
                         n_classes=num_labels,
                         n_layers=1,
                         gene_num=num_genes,
                         activation=F.relu,
                         dropout=0.1).to(device_train)
    
    offspring.load_state_dict(base_sd)
    
    return offspring
    

def create_offspring(population, fitness, rho, sigma):

    
    # Perform selection
    parents = random.choices(population,weights=fitness, k=rho) 
    
    # Perform crossover and mutation
    offspring = crossover_and_mutation(parents, sigma)
    
    
    return offspring


def GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, train_ids,
                test_ids,
                graph,
                batch_size,
                num_cells,
                num_genes,
                labels):
    
    #Calculate fitness of trained population

    fitness = [calc_loss(population[i],train_ids,graph,batch_size,num_cells,num_genes,labels) for i in range(pop_size)]
    
    print(f"--- -- Finished fitness evaluation, length: {len(fitness)}")
    
    #Create offspring population
    
    fitness_weighted = fitness
    offspring_population = [create_offspring(population, fitness_weighted, rho, sigma) for i in range(offspring_size)]
    
    print("--- -- Finished creating offspring population")
    
    #Evaluate fitness of offsprings 
    
    offspring_fitness = [calc_loss(offspring_population[i],test_ids,graph,batch_size,num_cells,num_genes,labels) for i in range(offspring_size)]
    
    print("--- -- Finished evaluating fitness of offspring population")
    
    # Combine fitness and population lists
    
    combined_fitness = fitness + offspring_fitness
    combined_population = population + offspring_population
    
    # sort and select population by their fitness values
    
    sorted_population = [pop for _, pop in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    sorted_fitness = [loss for loss, _ in sorted(zip(combined_fitness, combined_population), key=lambda pair: pair[0])]
    
    m = int(pop_size * elitist_level)
    new_population = sorted_population[0:m]
    
    # Fill up rest of population
    difference = pop_size - m
    remaining_population = list(set(sorted_population) - set(new_population))
    filler_population = random.sample(remaining_population, difference)
    
    # assemble new population and return
    new_population = new_population + filler_population
    
    return new_population, sorted_fitness

In [22]:
def calc_loss(model, train_ids,graph,batch_size,num_cells,num_genes,labels):
    model.eval()
    total_correct, total_unsure = 0, 0
    for nf in NeighborSampler(g=graph,
                              batch_size=batch_size,
                              expand_factor=num_cells + num_genes,
                              num_hops=1,
                              neighbor_type='in',
                              shuffle=True,
                              num_workers=8,
                              seed_nodes=train_ids.long()):
        nf.copy_from_parent()  # Copy node/edge features from the parent graph.
        with torch.no_grad():
            logits = model(nf).cpu()
        batch_nids = nf.layer_parent_nid(-1).type(torch.long)
        logits = nn.functional.softmax(logits, dim=1).numpy()
        label_list = labels.cpu()[batch_nids]
        for pred, label in zip(logits, label_list):
            max_prob = pred.max().item()
            if max_prob < 2 / num_labels:
                total_unsure += 1
            elif pred.argmax().item() == label:
                total_correct += 1

    return total_correct

# GA_Neural

In [23]:
def GA_Neural_train(population,
                    pop_size,
                    max_generations, 
                    Adam_epochs, GA_steps, 
                    offspring_size, elitist_level, rho,
                    learning_rate,
                   weight_decay,
                   batch_size,
                   num_cells,num_genes,graph,train_ids,test_ids,labels):
    
    print(f"Starting with population of size: {pop_size}")
    
    
    for k in range(max_generations):
        print(f"Currently in generation {k+1}")
        
        #SGD
        print(f"--- Starting Adam")
        
        # Sequential version
        #population = [SGD_training(population[i], SGD_steps, learning_rate, 0.9, train_loader) for i in range(pop_size)]
        for i in range(pop_size):
            train = Adam_Trainer(device_train = device_train,gnn_model = population[i],
                                 lr = learning_rate,weight_decay = weight_decay,Adam_epochs = Adam_epochs,
                                 batch_size = batch_size,num_cells = num_cells,
                                                        num_genes = num_genes,
                                                        num_labels = num_labels,
                                                        graph = graph,
                                                        train_ids = train_ids,
                                                        test_ids = test_ids,
                                                        labels = labels)
            train.fit()
        
        print(f"--- Finished Adam")
         
        # GA
        print(f"--- Starting GA")
        GA_start = time.time()
        sorted_fitness = []          # store the sorted fitness values to maybe use in data collection
        for i in range(0, GA_steps):
            
            sigma = 0.01 / (k+1)
            population, sorted_fitness = GA_training(population, pop_size, offspring_size, elitist_level, rho, sigma, train_ids,
                                                    test_ids,
                                                    graph,
                                                    batch_size,
                                                    num_cells,
                                                    num_genes,
                                                    labels)
        
        GA_end = time.time()
        print(f"--- Finished GA,Time:{(GA_end-GA_start)*1000}ms")
        
        
    print(f"Finished training process")
    return population

# Start training process
We have now defined the whole training algorithm. The next step is to actually perform training.

In [24]:
# Hyperparameters
#device_train = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device_train = torch.device('cpu')
batch_size = 500
pop_size = 5
max_generations = 100
Adam_epochs = 10
GA_steps = 1
offspring_size = 50
elitist_level = 0.6
rho = 3
learning_rate = 1e-3
in_feats = 400
n_hidden = 200
weight_decay = 5e-4

In [25]:
# Create population and start training process
population = [GNN(in_feats=in_feats,
                         n_hidden=n_hidden,
                         n_classes=num_labels,
                         n_layers=1,
                         gene_num=num_genes,
                         activation=F.relu,
                         dropout=0.1).to(device_train) for i in range(pop_size)]

# Train Process

In [27]:
Train_start = time.time()
trained_population = GA_Neural_train(population=population,
                                    pop_size = pop_size,
                                    max_generations=max_generations,
                                    Adam_epochs=Adam_epochs,GA_steps=GA_steps,
                                    offspring_size=offspring_size,elitist_level=elitist_level,rho=rho,
                                    learning_rate=learning_rate,
                               weight_decay = weight_decay,
                               batch_size = batch_size,
                               num_cells = num_cells,num_genes = num_genes,graph = graph,
                                     train_ids = train_ids,test_ids = test_ids,
                                     labels = labels)
Train_end = time.time()
print(f"All Time:{(Train_start-Train_end)*1000}ms")

Starting with population of size: 15
Currently in generation 1
--- Starting Adam
>>>>Epoch 0001: Train Acc 0.6292, Loss 1.9212, Test correct 235, Test unsure 0, Test Acc 0.6369
>>>>Epoch 0002: Train Acc 0.7016, Loss 1.4678, Test correct 247, Test unsure 0, Test Acc 0.6694
>>>>Epoch 0003: Train Acc 0.6712, Loss 1.2482, Test correct 224, Test unsure 0, Test Acc 0.6070
>>>>Epoch 0004: Train Acc 0.6069, Loss 1.1423, Test correct 196, Test unsure 0, Test Acc 0.5312
>>>>Epoch 0005: Train Acc 0.6671, Loss 1.1359, Test correct 218, Test unsure 0, Test Acc 0.5908
Epoch 0002, Train Acc 0.7016, Test Correct Num 247, Test Total Num 369, Test Unsure Num 0, Test Acc 0.6694
>>>>Epoch 0001: Train Acc 0.5805, Loss 1.2808, Test correct 197, Test unsure 0, Test Acc 0.5339
>>>>Epoch 0002: Train Acc 0.7395, Loss 1.1630, Test correct 262, Test unsure 0, Test Acc 0.7100
>>>>Epoch 0003: Train Acc 0.6725, Loss 1.0785, Test correct 224, Test unsure 0, Test Acc 0.6070
>>>>Epoch 0004: Train Acc 0.7172, Loss 0.999

>>>>Epoch 0001: Train Acc 0.0589, Loss 6.1446, Test correct 30, Test unsure 1, Test Acc 0.0813
>>>>Epoch 0002: Train Acc 0.4161, Loss 4.2172, Test correct 138, Test unsure 0, Test Acc 0.3740
>>>>Epoch 0003: Train Acc 0.4391, Loss 3.2855, Test correct 143, Test unsure 0, Test Acc 0.3875
>>>>Epoch 0004: Train Acc 0.4391, Loss 2.6626, Test correct 143, Test unsure 0, Test Acc 0.3875
>>>>Epoch 0005: Train Acc 0.4675, Loss 1.8523, Test correct 151, Test unsure 1, Test Acc 0.4092
Epoch 0005, Train Acc 0.4675, Test Correct Num 151, Test Total Num 369, Test Unsure Num 1, Test Acc 0.4092
--- Finished Adam
--- Starting GA
--- -- Finished fitness evaluation, length: 15
--- -- Finished creating offspring population
--- -- Finished evaluating fitness of offspring population
--- Finished GA,Time:96249.18174743652ms
Currently in generation 2
--- Starting Adam
>>>>Epoch 0001: Train Acc 0.1272, Loss 2.5949, Test correct 46, Test unsure 45, Test Acc 0.1247
>>>>Epoch 0002: Train Acc 0.7064, Loss 1.7760, 

>>>>Epoch 0005: Train Acc 0.7334, Loss 1.0283, Test correct 247, Test unsure 0, Test Acc 0.6694
Epoch 0005, Train Acc 0.7334, Test Correct Num 247, Test Total Num 369, Test Unsure Num 0, Test Acc 0.6694
>>>>Epoch 0001: Train Acc 0.2294, Loss 2.6521, Test correct 88, Test unsure 0, Test Acc 0.2385
>>>>Epoch 0002: Train Acc 0.7442, Loss 1.7227, Test correct 251, Test unsure 2, Test Acc 0.6802
>>>>Epoch 0003: Train Acc 0.5419, Loss 1.3273, Test correct 185, Test unsure 0, Test Acc 0.5014
>>>>Epoch 0004: Train Acc 0.4993, Loss 1.3052, Test correct 169, Test unsure 0, Test Acc 0.4580
>>>>Epoch 0005: Train Acc 0.5338, Loss 1.3613, Test correct 177, Test unsure 0, Test Acc 0.4797
Epoch 0002, Train Acc 0.7442, Test Correct Num 251, Test Total Num 369, Test Unsure Num 2, Test Acc 0.6802
>>>>Epoch 0001: Train Acc 0.7192, Loss 1.3448, Test correct 241, Test unsure 0, Test Acc 0.6531
>>>>Epoch 0002: Train Acc 0.6874, Loss 1.1953, Test correct 227, Test unsure 0, Test Acc 0.6152
>>>>Epoch 0003: Tra

>>>>Epoch 0002: Train Acc 0.6698, Loss 1.1547, Test correct 218, Test unsure 0, Test Acc 0.5908
>>>>Epoch 0003: Train Acc 0.7124, Loss 1.1008, Test correct 249, Test unsure 0, Test Acc 0.6748
>>>>Epoch 0004: Train Acc 0.7206, Loss 1.0952, Test correct 252, Test unsure 0, Test Acc 0.6829
>>>>Epoch 0005: Train Acc 0.7368, Loss 1.0103, Test correct 259, Test unsure 0, Test Acc 0.7019
Epoch 0005, Train Acc 0.7368, Test Correct Num 259, Test Total Num 369, Test Unsure Num 0, Test Acc 0.7019
>>>>Epoch 0001: Train Acc 0.5352, Loss 1.3981, Test correct 174, Test unsure 0, Test Acc 0.4715
>>>>Epoch 0002: Train Acc 0.6719, Loss 1.2265, Test correct 218, Test unsure 0, Test Acc 0.5908
>>>>Epoch 0003: Train Acc 0.7260, Loss 1.1480, Test correct 243, Test unsure 0, Test Acc 0.6585
>>>>Epoch 0004: Train Acc 0.6766, Loss 1.0971, Test correct 221, Test unsure 0, Test Acc 0.5989
>>>>Epoch 0005: Train Acc 0.7118, Loss 1.0377, Test correct 238, Test unsure 0, Test Acc 0.6450
Epoch 0003, Train Acc 0.7260,

>>>>Epoch 0001: Train Acc 0.6732, Loss 1.1131, Test correct 223, Test unsure 0, Test Acc 0.6043
>>>>Epoch 0002: Train Acc 0.7476, Loss 1.0545, Test correct 263, Test unsure 0, Test Acc 0.7127
>>>>Epoch 0003: Train Acc 0.7145, Loss 0.9904, Test correct 237, Test unsure 0, Test Acc 0.6423
>>>>Epoch 0004: Train Acc 0.7260, Loss 0.9419, Test correct 243, Test unsure 0, Test Acc 0.6585
>>>>Epoch 0005: Train Acc 0.7760, Loss 0.9223, Test correct 264, Test unsure 0, Test Acc 0.7154
Epoch 0005, Train Acc 0.7760, Test Correct Num 264, Test Total Num 369, Test Unsure Num 0, Test Acc 0.7154
>>>>Epoch 0001: Train Acc 0.5812, Loss 1.2187, Test correct 188, Test unsure 0, Test Acc 0.5095
>>>>Epoch 0002: Train Acc 0.6942, Loss 1.1761, Test correct 227, Test unsure 0, Test Acc 0.6152
>>>>Epoch 0003: Train Acc 0.6488, Loss 1.1338, Test correct 213, Test unsure 0, Test Acc 0.5772
>>>>Epoch 0004: Train Acc 0.6894, Loss 1.0846, Test correct 224, Test unsure 0, Test Acc 0.6070
>>>>Epoch 0005: Train Acc 0.7

...

>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0066, Test correct 311, Test unsure 0, Test Acc 0.8428
Epoch 0001, Train Acc 1.0000, Test Correct Num 311, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8428
--- Finished Adam
--- Starting GA
--- -- Finished fitness evaluation, length: 15
--- -- Finished creating offspring population
--- -- Finished evaluating fitness of offspring population
--- Finished GA,Time:83295.24827003479ms
Currently in generation 89
--- Starting Adam
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0042, Test correct 310, Test unsure 0, Test Acc 0.8401
Epoch 0001, Train Acc 1.0000, Test Correct Num 310, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8401
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0041, Test correct 311, Test unsure 0, Test Acc 0.8428
Epoch 0001, Train Acc 1.0000, Test Correct Num 311, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8428
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0074, Test correct 308, Test unsure 0, Test Acc 0.8347
Epoch 0001, Train Acc 1

>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0059, Test correct 314, Test unsure 0, Test Acc 0.8509
Epoch 0001, Train Acc 1.0000, Test Correct Num 314, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8509
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0059, Test correct 305, Test unsure 0, Test Acc 0.8266
Epoch 0001, Train Acc 1.0000, Test Correct Num 305, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8266
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0071, Test correct 309, Test unsure 0, Test Acc 0.8374
Epoch 0001, Train Acc 1.0000, Test Correct Num 309, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8374
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0055, Test correct 306, Test unsure 0, Test Acc 0.8293
Epoch 0001, Train Acc 1.0000, Test Correct Num 306, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8293
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0059, Test correct 307, Test unsure 0, Test Acc 0.8320
Epoch 0001, Train Acc 1.0000, Test Correct Num 307, Test Total Num 369, Test Unsure Num 0, T

>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0131, Test correct 305, Test unsure 0, Test Acc 0.8266
Epoch 0001, Train Acc 1.0000, Test Correct Num 305, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8266
--- Finished Adam
--- Starting GA
--- -- Finished fitness evaluation, length: 15
--- -- Finished creating offspring population
--- -- Finished evaluating fitness of offspring population
--- Finished GA,Time:89036.40556335449ms
Currently in generation 94
--- Starting Adam
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0102, Test correct 306, Test unsure 0, Test Acc 0.8293
Epoch 0001, Train Acc 1.0000, Test Correct Num 306, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8293
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0055, Test correct 307, Test unsure 0, Test Acc 0.8320
Epoch 0001, Train Acc 1.0000, Test Correct Num 307, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8320
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0070, Test correct 308, Test unsure 0, Test Acc 0.8347
Epoch 0001, Train Acc 1

>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0073, Test correct 308, Test unsure 0, Test Acc 0.8347
Epoch 0001, Train Acc 1.0000, Test Correct Num 308, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8347
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0104, Test correct 305, Test unsure 0, Test Acc 0.8266
Epoch 0001, Train Acc 1.0000, Test Correct Num 305, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8266
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0040, Test correct 311, Test unsure 0, Test Acc 0.8428
Epoch 0001, Train Acc 1.0000, Test Correct Num 311, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8428
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0088, Test correct 307, Test unsure 0, Test Acc 0.8320
Epoch 0001, Train Acc 1.0000, Test Correct Num 307, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8320
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0078, Test correct 305, Test unsure 0, Test Acc 0.8266
Epoch 0001, Train Acc 1.0000, Test Correct Num 305, Test Total Num 369, Test Unsure Num 0, T

>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0053, Test correct 301, Test unsure 0, Test Acc 0.8157
Epoch 0001, Train Acc 1.0000, Test Correct Num 301, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8157
--- Finished Adam
--- Starting GA
--- -- Finished fitness evaluation, length: 15
--- -- Finished creating offspring population
--- -- Finished evaluating fitness of offspring population
--- Finished GA,Time:83558.12048912048ms
Currently in generation 99
--- Starting Adam
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0045, Test correct 304, Test unsure 0, Test Acc 0.8238
Epoch 0001, Train Acc 1.0000, Test Correct Num 304, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8238
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0256, Test correct 308, Test unsure 0, Test Acc 0.8347
Epoch 0001, Train Acc 1.0000, Test Correct Num 308, Test Total Num 369, Test Unsure Num 0, Test Acc 0.8347
>>>>Epoch 0001: Train Acc 1.0000, Loss 0.0053, Test correct 311, Test unsure 0, Test Acc 0.8428
Epoch 0001, Train Acc 1